In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install flaml
!pip install mplcyberpunk

import matplotlib.pyplot as plt
import pathlib, os, random, mplcyberpunk
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flaml import AutoML

In [ ]:
dfTrain = pd.read_csv('/kaggle/input/data-science-london-scikit-learn/train.csv', header=None)
dfTest = pd.read_csv('/kaggle/input/data-science-london-scikit-learn/test.csv', header=None)

In [ ]:
dfTrain

In [ ]:
dfTrainLables = pd.read_csv('/kaggle/input/data-science-london-scikit-learn/trainLabels.csv', header=None)
dfTrain['Target'] = dfTrainLables.iloc[:, 0]

In [ ]:
dfTrain

In [ ]:
X = np.array(dfTrain.iloc[:, 0:40])
y = np.asarray(dfTrain.iloc[:, -1]).astype('int64')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = AutoML()

model.fit(X_train, y_train, task='classification', time_budget=10, metric='accuracy')

In [ ]:
print('Best ML Model:', model.best_estimator)
print('Best hyperparmeter config:', model.best_config)
print('Best Accuracy on validation data: %f'%(1 - model.best_loss))
print('Training duration of best run: %f s'%(model.best_config_train_time))

In [ ]:
X_new = np.array(dfTest.iloc[:, :])
y_pred = model.predict(X_new)

In [ ]:
y_pred

In [ ]:
class TFmodel(tf.keras.Model):
    def __init__(self, model_name):
        super(TFmodel, self).__init__()
        self.model_name = model_name
        
        self.D1 = Dense(64, activation='relu')
        self.B1 = BatchNormalization()
        self.D2 = Dense(64, activation='relu')
        self.D3 = Dense(32, activation='relu')
        self.D4 = Dense(32, activation='relu')
        self.D5 = Dense(32, activation='relu')
        self.Dr1 = Dropout(0.4)
        self.D6 = Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())
        
    def call(self, x):
        x = self.D1(x)
        x = self.B1(x)
        x = self.D2(x)
        x = self.D3(x)
        x = self.D4(x)
        x = self.D5(x)
        x = self.Dr1(x)
        y = self.D6(x)
        return y
    
    def __repr__(self):
        name = '{}_Model'.format(self.model_name)
        return name
    
modelForTF = TFmodel(model_name='Huang')

modelForTF.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = './TFmodel.ckpt'
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------------------------------------LoadingModel-------------------------------------------')
    modelForTF.load_weights(checkpoint_save_path)
    
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path, save_best_only=True, save_weights_only=True)

history = modelForTF.fit(X, y, epochs=50, batch_size=32, validation_split=0.3, callbacks=[cp_callback])

modelForTF.summary()

file = open('./TFmodelWeights.txt', 'w')
for v in modelForTF.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
    
file.close()

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(16, 8))
plt.style.use('cyberpunk')
plt.subplot(1, 2, 1)
plt.title('Training And Validation ACC')
plt.plot(acc, label='Training')
plt.plot(val_acc, label='Validation')
mplcyberpunk.add_glow_effects()
plt.legend()

plt.subplot(1, 2, 2)
plt.title('Training And Validation Loss')
plt.plot(loss, label='Training')
plt.plot(val_loss, label='Validation')
mplcyberpunk.add_glow_effects()
plt.legend()

plt.show()

In [ ]:
y_predForTFmodel = tf.argmax(modelForTF.predict(X_new), axis=1)
y_predForTFmodel

In [ ]:
dfForsub = pd.DataFrame({'Target': y_predForTFmodel})

In [ ]:
dfForsub

In [ ]:
dfForsub.to_csv('SubmissionForTest.csv')